# Exercise - SVM

The data set for this exercise is from the banking industry. It contains data about the home loans of 2,500 bank clients. Each row represents a single loan. The columns include the characteristics of the client who used a loan. This is a binary classification task: predict whether a loan will be bad or not (1=Yes, 0=No). This is an important task for banks to prevent bad loans from being issued.

## Description of Variables

The description of variables are provided in "Loan - Data Dictionary.docx"

## Goal

Use the **loan.csv** data set and build a model to predict **BAD**. 

Since you have a relatively small data set, I recommend using cross-validation to evaluate your accuracy.

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [3]:
#We will predict the "price" value in the data set:

loan = pd.read_csv("loan.csv")
loan.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,25900,61064.0,94714.0,DebtCon,Office,2.0,0.0,0.0,98.809375,0.0,23.0,34.565944
1,0,26100,113266.0,182082.0,DebtCon,Sales,18.0,0.0,0.0,304.852469,1.0,31.0,33.193949
2,1,50000,220528.0,300900.0,HomeImp,Self,5.0,0.0,0.0,0.000000,0.0,2.0,NaN
3,1,22400,51470.0,68139.0,DebtCon,Mgr,9.0,0.0,0.0,31.168696,2.0,8.0,37.952180
4,0,20900,62615.0,87904.0,DebtCon,Office,5.0,NaN,NaN,177.864849,NaN,15.0,36.831076


# Split data (train/test)

In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(loan, test_size=0.3)

# Data Prep

Perform your data prep here. You can use pipelines like we do in the tutorials. Otherwise, feel free to use your own data prep steps. Eventually, you should do the following at a minimum:<br>
- Separate inputs from target<br>
- Impute/remove missing values<br>
- Standardize the continuous variables<br>
- One-hot encode categorical variables<br>

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Separate the target variable 

In [16]:
train_y = train['BAD']
test_y = test['BAD']

train_inputs = train.drop(['BAD'], axis=1)
test_inputs = test.drop(['BAD'], axis=1)

##  Identify the numeric, binary, and categorical columns

In [17]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [18]:
numeric_columns

['LOAN',
 'MORTDUE',
 'VALUE',
 'YOJ',
 'DEROG',
 'DELINQ',
 'CLAGE',
 'NINQ',
 'CLNO',
 'DEBTINC']

In [19]:
categorical_columns

['REASON', 'JOB']

# Pipeline

In [20]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [21]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [22]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [23]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

array([[-0.31412013, -1.30301181, -0.86148829, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.45733454,  0.7398414 ,  0.58636192, ...,  1.        ,
         0.        ,  0.        ],
       [-1.10330939,  0.2001631 ,  0.18146318, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.21657988, -0.83000156, -0.82081329, ...,  0.        ,
         0.        ,  0.        ],
       [-0.46486414,  1.79196675,  1.36974799, ...,  0.        ,
         0.        ,  0.        ],
       [-0.31412013, -0.08740643, -0.21782887, ...,  0.        ,
         0.        ,  0.        ]])

In [24]:
train_x.shape

(1750, 20)

# Tranform: transform() for TEST

In [25]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

array([[ 0.06717356,  0.36706438,  0.32127798, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.32432512,  0.57631513,  0.42769944, ...,  0.        ,
         1.        ,  0.        ],
       [-0.33185472,  0.41209537,  0.12227549, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.8993616 , -0.51071616, -0.32326299, ...,  0.        ,
         0.        ,  0.        ],
       [-0.5446698 , -0.82706576, -0.8366813 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.5446698 , -0.06422056, -0.11380525, ...,  0.        ,
         0.        ,  0.        ]])

In [26]:
test_x.shape

(750, 20)

# Calculate the Baseline

In [27]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_y)

DummyClassifier(strategy='most_frequent')

In [28]:
from sklearn.metrics import accuracy_score

In [29]:
# This is the baseline Train Accuracy

dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_y, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.6034285714285714


In [30]:
# This is the baseline Test Accuracy

dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_y, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.5773333333333334


# Train an SVM model with linear kernel

In [31]:
from sklearn.svm import SVC
 
lin_svm = SVC(kernel="linear")

lin_svm.fit(train_x, train_y)

SVC(kernel='linear')

### Calculate the accuracy

In [32]:
from sklearn.metrics import accuracy_score

In [34]:
#Predict the train values
train_y_pred = lin_svm.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.7525714285714286

In [35]:
#Predict the test values
test_y_pred = lin_svm.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.7266666666666667

# Train an SVM model with poly kernel

In [36]:
from sklearn.svm import SVC

# You need to enter a value for gamma. Remember, gamma controls the shape of the bell curve for rbf
# You can also set it is as gamma='scale'. This will be the default option in future releases

pol_svm = SVC(kernel="poly", degree=3, coef0=1, C=10)

pol_svm.fit(train_x, train_y)

SVC(C=10, coef0=1, kernel='poly')

### Calculate the accuracy

In [37]:
#Predict the train values
train_y_pred = pol_svm.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8965714285714286

In [38]:
#Predict the test values
test_y_pred = pol_svm.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.8173333333333334

# Train an SVM model with rbf kernel

In [39]:
rbf_svm = SVC(kernel="rbf", C=10, gamma='scale')

rbf_svm.fit(train_x, train_y)

SVC(C=10)

### Calculate the accuracy

In [40]:
#Predict the train values
train_y_pred = rbf_svm.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.9177142857142857

In [41]:
#Predict the test values
test_y_pred = rbf_svm.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.8453333333333334

# Optional: try grid search on one of your SVM models

In [42]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 4 (2×2) combinations of hyperparameters
    {'C': [5, 15], 
     'gamma': [0.1, 0.2]}
  ]

rbf_svm = SVC(kernel="rbf", decision_function_shape='ovr')

# train across 5 folds, that's a total of 4*5=20 rounds of training 
grid_search = GridSearchCV(rbf_svm, param_grid, cv=5,
                           scoring='accuracy', return_train_score=True)

grid_search.fit(train_x, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [5, 15], 'gamma': [0.1, 0.2]}],
             return_train_score=True, scoring='accuracy')

In [43]:
grid_search.best_params_

{'C': 15, 'gamma': 0.2}

In [44]:
grid_search.best_estimator_

SVC(C=15, gamma=0.2)

In [45]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.8085714285714285 {'C': 5, 'gamma': 0.1}
0.8400000000000001 {'C': 5, 'gamma': 0.2}
0.8251428571428573 {'C': 15, 'gamma': 0.1}
0.8531428571428572 {'C': 15, 'gamma': 0.2}


In [46]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.030587,0.009869,0.008640,0.000937,5,0.1,"{'C': 5, 'gamma': 0.1}",0.791429,0.828571,0.797143,...,0.808571,0.015439,4,0.907143,0.909286,0.912143,0.913571,0.899286,0.908286,0.005022
1,0.028881,0.000687,0.008904,0.000148,5,0.2,"{'C': 5, 'gamma': 0.2}",0.828571,0.834286,0.834286,...,0.840000,0.009731,2,0.954286,0.965714,0.964286,0.961429,0.957143,0.960571,0.004295
2,0.031240,0.000905,0.007534,0.000139,15,0.1,"{'C': 15, 'gamma': 0.1}",0.800000,0.828571,0.814286,...,0.825143,0.016303,3,0.941429,0.939286,0.942857,0.942143,0.935000,0.940143,0.002836
3,0.032546,0.000532,0.008288,0.000082,15,0.2,"{'C': 15, 'gamma': 0.2}",0.854286,0.845714,0.860000,...,0.853143,0.005299,1,0.986429,0.991429,0.990714,0.988571,0.987143,0.988857,0.001948


In [48]:
final_model = grid_search.best_estimator_

test_predictions = final_model.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_predictions)

0.892